# Phase Slope Index method

### Purpose
This is a walk-through notebook on *Robustly Estimating the Flow Direction of Information in Complex Physical Systems* paper, by *Guido Nolte, Andreas Ziehe, Vadim V. Nikulin, Alois Schlögl, Nicole Krämer, Tom Brismar, and Klaus-Robert Müller*, implementated in Julia-Language (please see http://doc.ml.tu-berlin.de/causality/ and [Nolte et al. 2008](http://link.aps.org/abstract/PRL/v100/e234101)).

### Acknowledgement
This work was funded by the German Federal Ministry of Education and Research [(BMBF)](https://www.bmbf.de/) in the project ALICE III under grant ref. 01IS18049B.

## Setting up Julia Env.

In [ ]:
# Add packages
# import Pkg
# Pkg.add("FFTW")
# Pkg.add("DSP")
# Pkg.add("Plots")
# Pkg.add("MAT")
# Pkg.add("Einsum")

# imports for PSI
using Statistics: mean, std
using FFTW: fft!
using DSP: hanning
using Einsum

# imports for notebook
using Random: randperm
using MAT: matread
using Plots: plot, heatmap, cgrad
;

## Data
`mixed_data` four channcels, where:
- channels 1 and 2 are i.i.d. uniform [0, 1] noise
- channel 3 is delayed (by 1 sample) channel 1
- channel 4 is delayed (by 16 samples) channel 1 plus i.i.d. uniform [0, 0.2] noise 


In [ ]:
# data generation
n_channels = 4  # number of channels
n_samples = 65536  # = 2^16 number of data points measured in each channel
fs = 128  # sampling frequency
time_array = Array(range(1, step=1/fs, length=n_samples))

# mixed data
rand_data = randn(Float64, (n_samples+16, 1)) # 
cause_source = rand_data[16:n_samples+15]  # channel 1
random_source = randn(Float64, n_samples)  # channel 2
effect_source = rand_data[15:n_samples+14]  #channel 3
weak_effect = rand_data[1:n_samples] .- (randn(Float64, (n_samples, 1)) ./ 5)
mixed_data = hcat(cause_source, random_source, effect_source, weak_effect)


p1 = plot(time_array[1:64], mixed_data[1:64, :],
         title = "Mixed data",
         label = ["Cause" "Random" "Effect" "noisy Effect"],
         linecolor = ["red" "green" "blue" "magenta"])

plot(p1, layout=(1, 1), size=(800, 450))


## PSI

In [ ]:
# include("../src/PhaseSlopeIndex.jl")
using PhaseSlopeIndex

In [ ]:
? data2psi

In [ ]:
seglen = 128  # segment length
segshift = 0  # slide (shift) length
eplen = 256  # epoch length
method = "bootstrap"  # resampling method of estimation
nboot = 100  # number of bootstrap resampling epochs

@time psi, psi_se, psi_normed = data2psi(mixed_data, seglen,
                                         eplen=eplen, method=method, nboot=nboot)
;

In [ ]:
p1 = heatmap(psi, ticks=false, yflip = true,
             yticks=([1, 2, 3, 4], ["Ch1", "Ch2", "Ch3", "Ch4"]),
             xticks=([1, 2, 3, 4], ["Ch1", "Ch2", "Ch3", "Ch4"]),
             color=cgrad([:blue, :green, :red]), title="Phase Slope Index")

p2 = heatmap(psi_se, ticks=false, yflip = true,
             yticks=([1, 2, 3, 4], ["Ch1", "Ch2", "Ch3", "Ch4"]),
             xticks=([1, 2, 3, 4], ["Ch1", "Ch2", "Ch3", "Ch4"]),
             color=cgrad([:blue, :green, :red]), title="PSI standard error")

plot(p1, p2, layout=(1, 2), size=(750, 300))